In [10]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
train_tape = "Z:/rppg/ccnu_datatape_160x36x36_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_160x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_deep_phys(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            #yield i, (j-tf.reduce_mean(j, axis=0))/(tf.math.reduce_std(j, axis=0)+1e-6)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_deep_phys(train).cache(f'{tmp}/deepphys'), to_tf_deep_phys(valid).cache(f'{tmp}/deepphys')
train, valid = to_tf_deep_phys(train), to_tf_deep_phys(valid)

In [12]:
deepphys = DeepPhys_end_to_end()
deepphys.compile(optimizer='Nadam', loss='mse')
deepphys.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(deepphys, input_sig=[tf.TensorSpec([160, 36, 36, 3])])/160:.0f}')
deepphys.summary()

Flops per frame:52158718
Model: "deep_phys_end_to_end_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 1)                 532899    
Total params: 532,899
Trainable params: 532,899
Non-trainable params: 0
_________________________________________________________________


In [ ]:
deepphys.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/DeepPhys_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
deepphys.load_weights('../weights/DeepPhys_CCNU.h5')

In [13]:
deepphys.load_weights('../weights/DeepPhys_CCNU.h5')

In [14]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [15]:
eval_on_dataset(test_set_CCNU, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_CCNU_CCNU.h5')
get_metrics('../results/DeepPhys_CCNU_CCNU.h5')

{'Sliding window': {'MAE': 1.521, 'RMSE': 4.401, 'R': 0.9062},
 'Whole video': {'MAE': 0.951, 'RMSE': 2.926, 'R': 0.95207}}

In [16]:
eval_on_dataset(test_set_CCNU_rPPG, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_CCNU_CCNU_rPPG.h5')
get_metrics('../results/DeepPhys_CCNU_CCNU_rPPG.h5')

{'Sliding window': {'MAE': 1.757, 'RMSE': 4.873, 'R': 0.8768},
 'Whole video': {'MAE': 0.845, 'RMSE': 1.672, 'R': 0.98382}}

In [17]:
eval_on_dataset(test_set_PURE, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_CCNU_PURE.h5')
get_metrics('../results/DeepPhys_CCNU_PURE.h5')

{'Sliding window': {'MAE': 5.711, 'RMSE': 14.079, 'R': 0.81443},
 'Whole video': {'MAE': 2.799, 'RMSE': 8.312, 'R': 0.93726}}

In [18]:
eval_on_dataset(test_set_UBFC_rPPG2, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_CCNU_UBFC.h5')
get_metrics('../results/DeepPhys_CCNU_UBFC.h5')

{'Sliding window': {'MAE': 1.792, 'RMSE': 4.212, 'R': 0.97035},
 'Whole video': {'MAE': 1.06, 'RMSE': 1.505, 'R': 0.99666}}